# **Imports**

In [ ]:
import numpy as np 
import pandas as pd 
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import os
import PIL
from PIL import Image
import glob
import time
import matplotlib.image as mpimg

# **Loading Data**

In [ ]:
plant_df = pd.read_csv('/kaggle/input/plant-pathology-2021-fgvc8/train.csv').sort_values('image')

In [ ]:
plant_df

# **Data Exploration**

In [ ]:
plant_df['image'].size

In [ ]:
labels_count = plant_df['labels'].value_counts()

In [ ]:
print(labels_count)

In [ ]:
labels_count.sum()

In [ ]:
classes = list(labels_count.index)
print(classes)

In [ ]:
images = glob.glob('/kaggle/input/plant-pathology-2021-fgvc8/train_images/*.jpg')
for image in images[:100]:
    with open(image, 'rb') as file:
        img = Image.open(file)
        print(img.size)

In [ ]:
already_visited_labels = []
i = 0
j = 0
figure, axes = plt.subplots(nrows=12, ncols=2, figsize=(250, 250))
while len(already_visited_labels) < 12:
    if plant_df['labels'][i] not in already_visited_labels:
        already_visited_labels.append(plant_df['labels'][i])
        axes[j, 0].imshow(mpimg.imread(r'/kaggle/input/plant-pathology-2021-fgvc8/train_images/' + plant_df['image'][i]))
        axes[j, 1].text(0, 0, plant_df['labels'][i], fontsize=300)
        j += 1
    i += 1
[axi.axis('off') for axi in axes.ravel()]
figure.show()
    

In [ ]:
def plot_dynamic_image_augmentations(image_path):
    data_generators = [tf.keras.preprocessing.image.ImageDataGenerator(
                            rotation_range=20),
                       tf.keras.preprocessing.image.ImageDataGenerator(
                            width_shift_range=0.2),
                       tf.keras.preprocessing.image.ImageDataGenerator(
                            height_shift_range=0.2),
                       tf.keras.preprocessing.image.ImageDataGenerator(
                            horizontal_flip=True),
                       tf.keras.preprocessing.image.ImageDataGenerator(
                            zoom_range = 0.2),
                       tf.keras.preprocessing.image.ImageDataGenerator(
                            shear_range = 0.2),
                       tf.keras.preprocessing.image.ImageDataGenerator(
                            
                            rotation_range=20,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            horizontal_flip=True,
                            zoom_range = 0.2,
                            shear_range = 0.2,
                        )]
    data_generators_names = ['Rotation Range 20',
                             'Width Shift Range  0.2',
                             'Height Shift Range 0.2',
                             'Horizontal Flip',
                             'Zoom Range 0.2',
                             'Shear Range 0.2',
                             'All Augmentations'
                            ]
    fig, rows = plt.subplots(nrows=len(data_generators), ncols=2, figsize=(18,18))
    for i in range(len(data_generators)): 
        image = mpimg.imread(image_path)
        images = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        data_generators[i].fit(images)
        image_iterator = data_generators[i].flow(images)
        for j in range(2):
            rows[i, j].title.set_text(data_generators_names[i])
            rows[i, j].imshow(image_iterator.next()[0].astype('int'))
            rows[i, j].axis('off')
    fig.tight_layout()
    fig.show()



In [ ]:
plot_dynamic_image_augmentations('/kaggle/input/plant-pathology-2021-fgvc8/train_images/800113bb65efe69e.jpg')

# **Data Preprocessing**

In [ ]:
TRAIN_PATH = "../input/plant-pathology-2021-fgvc8/train_images/"
NUM_CLASSES =  len(classes)
HEIGHT,WIDTH = 224,224
BATCH_SIZE = 32
SEED = 143
SPLIT = 0.1

In [ ]:
# Image training and testing datasets
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1/255.,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split = SPLIT,
    zoom_range = 0.2,
    shear_range = 0.2,
)

train_dataset = datagen.flow_from_dataframe(
    plant_df,
    directory = TRAIN_PATH,
    x_col = "image",
    y_col = "labels",
    target_size = (HEIGHT,WIDTH),
    class_mode='categorical',
    batch_size = BATCH_SIZE,
    subset = "training",
    shuffle = True,
    seed = SEED,
    validate_filenames = True
)

validation_dataset = datagen.flow_from_dataframe(
    plant_df,
    directory = TRAIN_PATH,
    x_col = "image",
    y_col = "labels",
    target_size = (HEIGHT,WIDTH),
    class_mode='categorical',
    batch_size = BATCH_SIZE,
    subset = "validation",
    shuffle = True,
    seed = SEED,
    validate_filenames = True
)

In [ ]:
train_dataset[0][0][0].shape

In [ ]:
train_dataset[0]

In [ ]:
for i in range(5):
    plt.figure(i)
    plt.title(classes[int(np.where(train_dataset[i][1][0] == 1.0)[0])])
    plt.imshow(train_dataset[i][0][0])
    plt.axis('off')

In [ ]:
for i in range(5):
    plt.figure(i)
    plt.title(classes[int(np.where(validation_dataset[i][1][0] == 1.0)[0])])
    plt.imshow(validation_dataset[i][0][0])
    plt.axis('off')

# **AlexNet**

In [ ]:
"""model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(224,224,3)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(12, activation='softmax')
])"""

In [ ]:
model = tf.keras.models.load_model('../input/plantpathologymodels/models/plant_pathology2')
model.summary()

# **Training**

In [ ]:
"""def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)"""

In [ ]:
"""root_logdir = os.path.join(os.curdir, "logs\\fit\\")

run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)"""

In [ ]:
"""model.compile(loss='categorical_crossentropy', optimizer=tf.optimizers.SGD(lr=0.001), metrics=['accuracy'])
model.summary()"""

# **Evaluation**

In [ ]:
TEST_PATH = "../input/plant-pathology-2021-fgvc8/test_images/"

In [ ]:
test_plant_images = []
globbed_files = glob.glob("../input/plant-pathology-2021-fgvc8/test_images/*.jpg")
for image_file in globbed_files:
    test_plant_images.append(os.path.basename(image_file))
test_plant_df = pd.DataFrame(test_plant_images, columns=['image']).sort_values('image').reset_index().drop('index', axis=1)

In [ ]:
test_plant_df['labels'] = 'None'

In [ ]:
print(test_plant_df)

In [ ]:
datagen_test = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1/255.)

test_dataset = datagen.flow_from_dataframe(
    test_plant_df,
    directory = TEST_PATH,
    x_col = "image",
    y_col = "labels",
    target_size = (HEIGHT,WIDTH),
    class_mode='categorical',
    batch_size = 1,
    shuffle = True,
    seed = SEED,
    validate_filenames = True
)

In [ ]:
predictions = model.predict(test_dataset)

In [ ]:
def convert_predictions_to_labels(predictions_list):
    labels_list = []
    for i in range(len(predictions_list)):
        labels_list.append(classes[np.argmax(predictions[i])])
    return labels_list
        

In [ ]:
print(predictions)
print(convert_predictions_to_labels(predictions))

In [ ]:
test_plant_df['labels'] = convert_predictions_to_labels(predictions)

In [ ]:
print(test_plant_df)

In [ ]:
test_plant_df.to_csv('./submission.csv', index=False)

In [ ]:
data = pd.read_csv('./submission.csv').sort_values('image')
print(data)